## 读取数据

In [31]:
import torch
from torch import nn
import gzip
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    """加载MNIST数据集"""
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte.gz')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte.gz')

    with gzip.open(labels_path, 'rb') as lbpath:
        struct.unpack('>II', lbpath.read(8))
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        struct.unpack('>IIII', imgpath.read(16))
        images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels),28,28)

    return images, labels



# 数据集划分
def data_split(images, labels, ratio):
    
    total_len = images.shape[0]
    offset = int(total_len * ratio)

    val_img = images[:offset][:]
    val_lb = labels[:offset]

    train_img = images[offset:][:]
    train_lb = labels[offset:]

    return train_img, train_lb, val_img, val_lb    

# 读取训练集和测试集数据
[images, labels] = load_mnist('./MNIST', kind='train')
[test_img, test_lb] = load_mnist('./MNIST',kind='test')
train_img, train_lb, val_img, val_lb = data_split(images, labels, 1/6)

# 对标签进行热编码
one_hot_train_lb = np.eye(10)[train_lb]
one_hot_val_lb = np.eye(10)[val_lb]
one_hot_test_lb= np.eye(10)[test_lb]

# 打印查看数据集格式
print('训练集图像格式为:', train_img.shape, '训练集标签格式为:', train_lb.shape,'热编码训练集标签格式为:', one_hot_train_lb.shape)
print('验证集图像格式为:', val_img.shape, '验证集标签格式为:', val_lb.shape,'热编码验证集标签格式为:', one_hot_val_lb.shape)
print('测试集图像格式为:', test_img.shape, '测试集标签格式为:', test_lb.shape,'热编码测试集标签格式为:', one_hot_test_lb.shape)


训练集图像格式为: (50000, 28, 28) 训练集标签格式为: (50000,) 热编码训练集标签格式为: (50000, 10)
验证集图像格式为: (10000, 28, 28) 验证集标签格式为: (10000,) 热编码验证集标签格式为: (10000, 10)
测试集图像格式为: (10000, 28, 28) 测试集标签格式为: (10000,) 热编码测试集标签格式为: (10000, 10)


## 定义网络

In [24]:
from torchvision.transforms import v2 
trans=v2.Compose([
    v2.RandomResizedCrop(size=(28,28),antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomHorizontalFlip(p=0.5)
])
train_img=torch.tensor(train_img)
new_X=trans(train_img)
new_X=torch.cat((train_img,new_X),dim=0)
new_Y=torch.tensor(one_hot_train_lb)
new_Y=torch.cat((new_Y,new_Y),dim=0)
train_img=new_X
one_hot_train_lb=new_Y

/tmp/ipykernel_426695/860925982.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_img=torch.tensor(train_img)


In [32]:
class NeuralNetwork(nn.Module):
    def __init__(self):

        super().__init__()
        self.w1=nn.Linear(28*28, 512)
        self.w2=nn.Linear(512, 512)
        self.w3=nn.Linear(512, 10)
        self.drop=nn.Dropout(p=0.5)
        self.relu=nn.ReLU()


    def forward(self, x):
        x = x.view(len(x),-1)
        x = self.w1 (x)
        x = self.relu(x)
        x= self.drop(x)
        x = self.w2 (x)
        x = self.relu(x)
        self.drop(x)
        x = self.w3 (x)
        return x

model = NeuralNetwork()

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in model.modules():
    if isinstance(i,nn.Linear):
        nn.init.xavier_uniform_(i.weight)

## 定义训练

In [ ]:

batch_size = 100
epochs = 100
batch_num=int(train_img.shape[0]/batch_size)
size = len(train_img)

model.train()
for t in range(epochs):
    
    correct=0.
    train_mean_loss=0.

    for batch in range(batch_num):
        X=train_img[batch*batch_size:(batch+1)*batch_size,]
        y=one_hot_train_lb[batch*batch_size:(batch+1)*batch_size,:]

        X=torch.tensor(X, dtype=torch.float32)
        y=torch.tensor(y, dtype=torch.float32)

        # Compute prediction and loss
        pred = model(X).type(torch.float)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).mean().item()
        train_mean_loss+= loss.item()

    train_mean_loss /= batch_num
    correct /= batch_num
    
    print(f" Epoch:{t+1}, loss: {train_mean_loss:>8f},  Accuracy: {(100*correct):>0.1f}%")


 Epoch:1, loss: 4.368667,  Accuracy: 79.8%
 Epoch:2, loss: 0.815133,  Accuracy: 83.2%
 Epoch:3, loss: 0.686101,  Accuracy: 85.1%


## 定义测试

In [ ]:
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
        X=torch.tensor(test_img, dtype=torch.float32)
        y=torch.tensor(one_hot_test_lb, dtype=torch.float32)
        pred = model(X)
        test_loss = np.mean(loss_fn(pred, y).item())
        correct = (pred.argmax(1) == y.argmax(1)).type(torch.float).mean().item()

print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
